- I ignored stop-words because I would like to use the dynamic list approach from the Saif et al. paper.
- The spellchecker is commented out because anything that didn't match its dictionary was replaced with None.
- The translate library is supposed to handle over 100 languages, but did not work on my small sample
- After running this, only 2 tweets were completely removed (and therefore replaced with 'cleaned'

In [3]:
!pip install contractions

     ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
     - -------------------------------------- 10.2/289.9 kB ? eta -:--:--
     ------- ----------------------------- 61.4/289.9 kB 812.7 kB/s eta 0:00:01
     -------------------------------------  286.7/289.9 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 289.9/289.9 kB 2.2 MB/s eta 0:00:00


In [12]:
!pip install emoji

  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/96/c6/0114b2040a96561fd1b44c75df749bbd3c898bf8047fb5ce8d7590d2dee6/emoji-2.8.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/358.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/358.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/358.9 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/358.9 kB 393.8 kB/s eta 0:00:01
   -------------------------- ------------- 235.5/358.9 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 358.9/358.9 kB 2.2 MB/s eta 0:00:00


In [20]:
!pip install pyspellchecker

     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.4 MB 187.9 kB/s eta 0:00:19
      --------------------------------------- 0.1/3.4 MB 299.4 kB/s eta 0:00:12
     - -------------------------------------- 0.2/3.4 MB 706.2 kB/s eta 0:00:05
     ------- -------------------------------- 0.6/3.4 MB 2.4 MB/s eta 0:00:02
     --------------------------- ------------ 2.4/3.4 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------  3.4/3.4 MB 9.9 MB/s eta 0:00:01
     ---------------------------------------- 3.4/3.4 MB 9.1 MB/s eta 0:00:00


In [45]:
pip install translate

  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/31/58/e3b3dd6bb2ab7404f1f4992e2d0e6926ed40cef8ce1b3bbefd95877499e1/lxml-4.9.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
    --------------------------------------- 0.1/3.8 MB 656.4 kB/s eta 0:00:06
   -- ------------------------------------- 0.2/3.8 MB 1.8 MB/s eta 0:00:02
   ------------ --------------------------- 1.2/3.8 MB 7.0 MB/s eta 0:00:01
   ------------------------------ --------- 2.9/3.8 MB 13.1 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 16.1 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 14.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
import spacy
import contractions
import emoji
import html

# from spellchecker import SpellChecker
from translate import Translator


In [2]:
# initialize spellchecker
# spell = SpellChecker()

In [3]:
# initialize translator
translator= Translator(to_lang="en")

In [29]:
data = pd.read_csv('train_tweets.csv')
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
def has_non_english(text):
    for char in text:
        # Check if the character is a letter and not in the ASCII range for English letters
        if char.isalpha() and ord(char) > 127:
            return True
    return False

In [6]:
# def spell_check(text):
#     words = text.split()
#     corrected_words = [spell.correction(word) for word in words]
#     corrected_text = ' '.join(corrected_words)
#     return corrected_text

In [30]:
def clean_text(text):
    # expand contractions
    cleaned_text = contractions.fix(text)
    # remove usernames
    cleaned_text = cleaned_text.replace('@user', '')
    # replace emojis with descriptive text
    cleaned_text = emoji.demojize(cleaned_text)
    # replace html characters with symbols
    cleaned_text = html.unescape(cleaned_text)
    # translate non-English languages
    if has_non_english(cleaned_text) == True:
        cleaned_text = translator.translate(cleaned_text) #doesn't work as intended on non-major languages
    # remove special characters
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', cleaned_text)
    # convert to all lowercase characters
    cleaned_text = cleaned_text.lower()
    if type(cleaned_text) == None:
        cleaned_text = 'cleaned'
    if cleaned_text.isspace():
        cleaned_text = 'cleaned'
    return cleaned_text
    

In [31]:
data['tweet'] = data['tweet'].apply(clean_text)

In [32]:
data.head()

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cannot use because ...
2,3,0,bihday your majesty
3,4,0,model i love you take with you all the time ...
4,5,0,factsguide society now motivation


In [33]:
data.to_csv('cleaned_data.csv', index=False, header=False)

In [35]:
# sum all the 'cleaned' cells
sum_cleaned_cells = 0
for row in data.values:
    for cell in row:
        if cell == 'cleaned':
            sum_cleaned_cells += 1

print("Sum of cells with 'cleaned':", sum_cleaned_cells)

Sum of cells with 'cleaned': 2
